---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="8a0dd99f" -->
# Table des matières
1. [Préparation et classification des données](#préparation-et-classification-des-données)
1. [L'interprétation des erreurs de classification](#linterprétation-des-erreurs-de-classification)
  1. [Les types d'erreurs de classification](#les-types-derreurs-de-classification)
  1. [Importances inégales des erreurs de classification](#importances-inégales-des-erreurs-de-classification)
1. [La matrice de confusion](#la-matrice-de-confusion)
    1. [La matrice de confusion est un outil d'aide à la détection et à l'interprétation des erreurs](#la-matrice-de-confusion-est-un-outil-daide-à-la-détection-et-à-linterprétation-des-erreurs)
1. [Les métriques de classification](#les-métriques-de-classification)
  1. [L'exactitude (*accuracy*)](#lexactitude-iaccuracyi)
  1. [La précision, le rappel, la sensibilité, la spécificité et la mesure F1](#la-précision-le-rappel-la-sensibilité-la-spécificité-et-la-mesure-f1)
    1. [La précision](#la-précision)
    1. [Le rappel (*recall*) ou la sensibilité](#le-rappel-irecalli-ou-la-sensibilité)
    1. [La spécificité](#la-spécificité)
    1. [La mesure F1 (*F1-Score*)](#la-mesure-f1-if1-scorei)
1. [Les métriques globales de classification](#les-métriques-globales-de-classification)
  1. [La courbe ROC (*Receiver Operator Characteristic*)](#la-courbe-roc-ireceiver-operator-characteristici)
  1. [La courbe Précision-Rappel](#la-courbe-précision-rappel)
  1. [Quelle métrique globale choisir pour sélectionner un classificateur parmi plusieurs?](#quelle-métrique-globale-choisir-pour-sélectionner-un-classificateur-parmi-plusieurs)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split

sns.set(color_codes=True)


seed = 42
np.random.seed(seed)



<!-- #region id="c36a7979" -->
Dans ce module, nous allons discuter de plusieurs métriques utilisées pour évaluer la qualité d'une classification.
Elles sont également utilisées pour comparer divers modèles entre eux afin de choisir le meilleur. (Nous aborderons
le sujet de la sélection de modèles dans le module sur la méthodologie.)

Pour ce faire, nous allons utiliser le jeu de données IRIS qui contient les mesurations de trois espèces
d'iris; Versicolor, Virginica et Setosa. La figure suivante montre une exemple de chaque espèce ainsi que les
mensurations contenues dans le jeu de données (longeurs et largeurs des pétales et des sépales). Ce jeu de données
est très utilisé en apprentissage automatique et en statistiques lorsqu'il est question de classification.

<!-- #endregion -->

<!-- #region id="c088db12" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/iris-flowers.jpeg"  width="500" />
    <div>
    <font size="0.5">Image Source: https://www.ubuntupit.com/best-machine-learning-datasets-for-practicing-applied-ml/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="73cb7666" -->
Nous allons classifier les iris du jeu de données en utilisant la régression logistique. Les résultats de
classification seront ensuite utilisés pour calculer les différentes métriques.
<!-- #endregion -->

<!-- #region id="8f923dfc" -->
# <a id=préparation-et-classification-des-données>Préparation et classification des données</a>
<!-- #endregion -->



In [ ]:
# Lecture des données

iris = datasets.load_iris()
X = iris.data
y = iris.target

noms = iris.target_names

n_classes = noms.shape[0]



<!-- #region id="e3ed05f4" -->
Génération des ensembles d'entraînement ($80~\%$ des données) et de test ($20~\%$ restants).
On utilise un échantillonnage stratifié afin de respecter la distribution des espèces d'iris dans chaque ensemble. 
On ne normalise pas les données dans ce qui suit puisqu'elles sont toutes du même ordre de grandeur.
<!-- #endregion -->



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=seed
)


In [ ]:
# Entraînement du classificateur avec les données d'entraînement

clf = LogisticRegression()
clf.fit(X_train, y_train)

# Prédictions des espèces d'iris pour l'ensemble de test

y_pred = clf.predict(X_test)



<!-- #region id="1ffdb300" -->
Dans ce qui suit, nous allons présenter les diverses métriques, puis les utiliser avec les
valeurs prédites $y_{\text{pred}}$ et les vraies valeurs (*ground truth*) $y_{\text{test}}$.
<!-- #endregion -->

<!-- #region id="f946c9d6" -->
# <a id=linterprétation-des-erreurs-de-classification>L'interprétation des erreurs de classification</a>
<!-- #endregion -->

<!-- #region id="e685c611" -->
## <a id=les-types-derreurs-de-classification>Les types d'erreurs de classification</a>
<!-- #endregion -->

<!-- #region id="fc40c933" -->
Prenons le cas le plus simple où l'on fait une classification selon deux classes. En général, on observe
que plusieurs prédictions sont correctes et d'autres non. Dans ce cas, il y a quatre résultats possibles.
Pour simplifier l'écriture, on va utiliser les classes $\{\text{Négatif}, \text{Positif}\}$. On aurait pu aussi employer les classes $\{0, 1\}$, $\{A, B\}$, $\{\text{Chien}, \text{Chat}\}$, etc.

On se sert de la notation suivante DE (pour Diagnostic-Estimation) où E indique le résultat estimé $\{\text{Négatif}, \text{Positif}\}$ et D indique le diagnostic après vérification $\{\text{Vrai}, \text{Faux}\}$.

Les quatres résultats possibles sont:

- Bonnes prédictions:
    - vrai positif (VP): prédiction de la classe positive **et** observation de la classe positive,
    - vrai négatif (VN): prédiction de la classe négative **et** observation de la classe négative.

- Mauvaises prédictions
    - faux positif (FP): prédiction de la classe positive **mais** observation de la classe négative,
    - faux négatif (FN): prédiction de la classe négative **mais** observation de la classe positive.
<!-- #endregion -->

<!-- #region id="2c1ec835" -->
## <a id=importances-inégales-des-erreurs-de-classification>Importances inégales des erreurs de classification</a>
<!-- #endregion -->

<!-- #region id="1de29224" -->
L'importance de chaque résultat dépend de la situation où un test (c.-à-d. une classification) est requis.
Prenons le cas d'un test de dépistage du cancer chez un patient. Il y a quatre éventualités
possibles. Le pire diagnostic est un faux négatif. Le patient, soulagé qu'on n'ait rien trouvé, retourne chez lui.
Malheureusement, il devra revenir plus tard lorsque son cancer se sera aggravé.

- Bonnes prédictions:
    - vrai positif (VP): le patient a le cancer (après vérification) et il est traité immédiatement!
    - vrai négatif (VN): le patient n'a pas le cancer.

- Mauvaises prédictions:

    - faux positif (FP): fausse alarme. Le patient n'a pas le cancer (après vérification),
    - faux négatif (FN): le patient a un cancer non détecté qui risque de s'empirer...

<!-- #endregion -->
<!-- #region id="d87e5a44" -->
Affichons les nombres de bonnes et mauvaises prédictions pour chaque espèce d'iris.
<!-- #endregion -->



In [ ]:
max_length = max(len(nom) for nom in noms)
print("Classe     VP VN FP FN")
for i in range(n_classes):
    VN, FP, FN, VP = confusion_matrix(y_test == i, y_pred == i).ravel()
    print("%s %2d %2d %2d %2d" % (noms[i].ljust(max_length), VP, VN, FP, FN))



<!-- #region id="350f516e" -->
# <a id=la-matrice-de-confusion>La matrice de confusion</a>
<!-- #endregion -->

<!-- #region id="50b5b270" -->
La matrice de confusion est un outil facilitant la visualisation des résultats de classification.

Voici un exemple de matrice de confusion à deux classes.
Comme dans l'exemple précédent, la présence d'une maladie doit être détectée; un test positif indique la présence de la maladie.
Les bonnes prédictions se retrouvent le long de la diagonale principale (gauche $\rightarrow$ droite).
<!-- #endregion -->

<!-- #region id="9acbe98c" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/sensibilité-vs-spécificité.png"  width="350" />
    <div>
    <font size="0.5">Image Source: https://fr.wikipedia.org/wiki/Sensibilité_et_spécificité</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="3d901b2b" -->
La figure suivante montre un exemple de test
de détection de pourriels (*SPAM*).
Le test a été effectué sur un ensemble contenant 100 courriels et 100 pourriels (précédemment identifiés par un lecteur averti).
Les résultats montrent que le classificateur fonctionne plutôt bien; seules quelques erreurs apparaissent dans les
cases hors de la diagonale principale.

<!-- #endregion -->

<!-- #region id="fbb9429d" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/confusion-matrix.png"  width="400" />
    <div>
    <font size="0.5">Image Source: https://fr.wikipedia.org/wiki/Matrice_de_confusion</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="9b15fd2d" -->
La figure suivante montre un exemple de matrice de confusion à onze classes. Chacune correspond au nom d'un
quartier dans la ville de Charlottesville en Virginie (États-Unis). Les vraies valeurs
(*true labels*) sont affichées le long des lignes. Les valeurs prédites sont affichées le long des colonnes.

Il est important de noter que les matrices de confusion sont rarement symmétriques.
Une carte de chaleur (*heatmap*) est souvent superposée à une matrice de confusion afin de
rendre plus visibles ses valeurs importantes (comme le long de la diagonale principale). De plus, il est courant
de normaliser les valeurs le long de l'axe associé aux vraies valeurs afin que la somme donne $100~\%$ pour chaque classe.

<!-- #endregion -->

<!-- #region id="9b822651" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/confusion-matrix2.png"  width="500" />
    <div>
    <font size="0.5">Image Source: https://ccri.com/better-visualizations-with-ccris-new-plotting-library/</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="d3336d68" -->
Pour afficher la matrice de confusion de notre classificateur, on utilise une approche un contre tous (*one versus all*). Par exemple dans le cas de l'espèce Setosa, il y a deux classes: Setosa et non Setosa. L'échelle de couleur correspond au nombre de fleurs dans chaque case.
<!-- #endregion -->



In [ ]:
fig = plt.subplots(1, 3, figsize=(15, 4))
plt.subplots_adjust(left=0.13, right=0.93, top=1.0, bottom=0.27, wspace=0.3, hspace=0.3)

for i in range(n_classes):
    ax = plt.subplot(1, 3, i + 1)
    cm = confusion_matrix(y_test == i, y_pred == i)

    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=["autre", noms[i]]
    )
    disp.plot(ax=ax, cmap="Blues")
    plt.tick_params(axis=u"both", which=u"both", length=0)
    plt.grid(visible=False)



<!-- #region id="f70d12e2" -->
On voit dans la matrice de gauche qu'il y a effectivement les valeurs que nous avons calculées précédemment, soit $\text{VP}=10$, $\text{VN}=20$, $\text{FP}=0$, $\text{FN}=0$.
Les iris setosa sont parfaitement classifiés. Ce n'est pas le case des deux autres espèces.
<!-- #endregion -->

<!-- #region id="efb10deb" -->
Il est aussi possible d'afficher la matrice de confusion combinant les 3 classes d'iris.
<!-- #endregion -->



In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=noms)
disp.plot(cmap="Blues")
plt.tick_params(axis=u"both", which=u"both", length=0)
plt.grid(visible=False)



<!-- #region id="9b4a3480" -->
Selon la diagonale principale, on a 10 setosa, 9 versicolor et 10 virginica correctement classifiées; ces nombres
correspondent aux vrais positifs dans chacune des trois matrices précédentes.
<!-- #endregion -->

<!-- #region id="0ed6c4d6" -->
## <a id=la-matrice-de-confusion-est-un-outil-daide-à-la-détection-et-à-linterprétation-des-erreurs>La matrice de confusion est un outil d'aide à la détection et à l'interprétation des erreurs</a>
<!-- #endregion -->

<!-- #region id="c33adb90" -->
La matrice de confusion donne beaucoup d'indices pour identifier les erreurs de classification; elles sont toutes situées
hors de la diagonale principale.

Les Setosa ont bien été classées; aucune n'a été confondue avec des Versicolor ou des Virginica.
Par contre, on voit que six Virginica ont été confondues avec des Versicolor et trois Versicolor ont été
confondues avec des Virginica.

> À noter que la matrice de confusion n'est pas symétrique.

On pourrait améliorer la classification en ajoutant de nouvelles images de Virginica et de Versicolor dans la base de
données. Le classificateur apprendrait à mieux les différencier. C'est une façon de procéder ayant fait ses preuves.

<!-- #endregion -->

<!-- #region id="162007ee" -->
# <a id=les-métriques-de-classification>Les métriques de classification</a>
<!-- #endregion -->

<!-- #region id="fbe92933" -->
## <a id=lexactitude-iaccuracyi>L'exactitude (*accuracy*)</a>
<!-- #endregion -->

<!-- #region id="9099a427" -->
C'est la métrique la plus intuitive de toutes. Elle représente la proportion de prédictions correctes (à la fois vraies
positives et vraies négatives) parmi l'ensemble des prédictions.

Dans le cas d'une classification selon deux classes, on la calcule comme suit

$$\text{Exactitude} = \dfrac{\text{VP} + \text{VN}}{\text{VP} + \text{VN} + \text{FP} + \text{FN}}.$$

Dans le cas plus général d'une classification multiclasses, le calcul utilise la matrice de
confusion $M$ associée comme suit

$$\begin{align}
\text{Exactitude} &= \dfrac{\text{Somme des valeurs le long de la diagonale principale}}{\text{Somme de toutes les valeurs}} \\
  &= \dfrac{\sum\limits_{i=1}^N M_{i,i}}{\sum_{i=1}^{N}\sum_{j=1}^{N} M_{i,j}}.
\end{align}$$

L'exactitude est toutefois sensible au débalancement des classes lors d'une classification. Supposons que l'on
veuille classifier des images de chats et de chiens et que l'on dispose d'un  jeu de données
d'entraînement contenant $99~\%$ d'images chats et seulement $1~\%$ d'images de chiens. Si on utilise un classificateur
stupide prédisant toujours la classe la plus populeuse alors il prédira toujours que des chats. Il ne se trompera
qu'avec les images de chiens, soit $1~\%$ du temps. Un tel classificateur aura une exactitude de $99~\%$ même s'il
ne fait aucune classification! C'est la principale faiblesse de la métrique d'exactitude.

Il est correct d'utiliser la métrique d'exactitude dans les situations où toutes les classes sont balancées.
<!-- #endregion -->

<!-- #region id="5f3d3d85" -->
## <a id=la-précision-le-rappel-la-sensibilité-la-spécificité-et-la-mesure-f1>La précision, le rappel, la sensibilité, la spécificité et la mesure F1</a>
<!-- #endregion -->

<!-- #region id="fdd81383" -->
Ces nouvelles métriques ajoutent un niveau de subtilité dans l'évaluation des performances d'un classificateur.

Supposons que vous avez développé un test de dépistage pour une maladie particulière. Votre test est sous évaluation par les
instances de la santé. Il est appliqué à des sujets sains et des sujets malades pour lesquels l'état de santé est
parfaitement connu.

La figure suivante montre les résultats.
<!-- #endregion -->

<!-- #region id="f9d7879e" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/precision-recall-and-others.png" width="400" />
    <div>
    <font size="0.5">Image Source: https://fr.wikipedia.org/wiki/Précision_et_rappel/</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="22de79d9" -->
La colonne de gauche (éléments pertinents) correspond aux sujets malades et celles de droite aux sujets sains.
Dans la colonne de gauche, les vrais positifs sont des cas réels dépistés, les faux négatifs sont des cas réels ignorés.
<!-- #endregion -->

<!-- #region id="00b76002" -->
### <a id=la-précision>La précision</a>
La précision est la proportion des tests positifs qui correspondent réellement à des sujets malades.

$$\text{Précision} = \dfrac {\text{VP}}{\text{VP} + \text{FP}}$$
Si votre test identifie 100 sujets positifs alors qu'en réalité, seuls 60 d'entre eux le sont vraiment alors
votre test a une précision de $60~\%$. Il génère également $40~\%$ de fausses alarmes!
<!-- #endregion -->

<!-- #region id="c8988908" -->
### <a id=le-rappel-irecalli-ou-la-sensibilité>Le rappel (*recall*) ou la sensibilité</a>
<!-- #endregion -->

<!-- #region id="6b2ec75a" -->
Le rappel est la proportion des sujets malades détectés par votre test et qui le sont réellement.

$$\text{Rappel} = \dfrac{\text{VP}}{\text{VP} + \text{FN}}$$
Supposons qu'on vous demande de tester une cohorte de 100 sujets malades et que le test n'en détecte que
70 parmi eux; il a alors un rappel de $70~\%$.
<!-- #endregion -->

<!-- #region id="918a3020" -->
### <a id=la-spécificité>La spécificité</a>
<!-- #endregion -->

<!-- #region id="afd7372c" -->
Ce que le rappel mesure pour les sujets malades, la spécificité le mesure pour les sujets sains. Ainsi,
la spécificité est la proportion des sujets sains détectés par votre test et qui le sont réellement.

$$\text{Spécificité} = \dfrac{\text{VN}}{\text{VN} + \text{FP}}$$
Supposons qu'on vous demande de tester une cohorte de 100 sujets sains et que le test détecte
30 sujets malades parmi eux; il a alors une spécificité de $70~\%$.
<!-- #endregion -->

<!-- #region id="be1d0dcc" -->
### <a id=la-mesure-f1-if1-scorei>La mesure F1 (*F1-Score*)</a>
<!-- #endregion -->

<!-- #region id="f38fd98e" -->
La précision et le rappel donnent de l'information sur les performances d'un test lorsqu'il détecte la classe d'intérêt.
Dans la pratique, il arrive que l'on doive choisir l'une ou l'autre de ces métriques alors que les deux sont pertinentes.
Laquelle choisir?

La métrique F1 offre un compromis entre les deux en calculant leur moyenne harmonique selon

$$F_{1}=2\cdot \dfrac {(\text{Précision}\cdot \text{Rappel})}{(\text{Précision}+\text{Rappel})}$$
<!-- #endregion -->

<!-- #region id="1d1ca5bd" -->
Le rapport de classification (`classification_report`), que nous utilisons depuis un moment déjà, présente les statistiques d'exactitude, de précision, de rappel et de F1. Calculons-le pour notre problème.
<!-- #endregion -->



In [ ]:
print(classification_report(y_test, y_pred, target_names=noms))



<!-- #region id="d28dcdfa" -->
Les résultats en précision montrent que le classificateur performe mieux avec les fleurs de type Setosa et Versicolor; $100~\%$ des prédictions de ces iris étaient correctes. Les résultats en rappel montrent que le classificateur performe mieux avec les fleurs de type Setosa et Virginica; $100~\%$ de ces iris ont été identifiés.

On observe une exactitude (`accuracy`) de $97~\%$, ce qui est très bon.
<!-- #endregion -->

<!-- #region id="54c4c148" -->
# <a id=les-métriques-globales-de-classification>Les métriques globales de classification</a>
<!-- #endregion -->

<!-- #region id="028fc3a2" -->
Pour simplifier la discussion, nous allons faire l'hypothèse, dans ce qui suit, que le problème de classification n'implique que deux classes. En plus d'indiquer la classe gagnante, plusieurs classificateurs peuvent aussi donner la probabilité de chaque classe. Par défaut, la classe gagnante est celle pour laquelle la probabilité est supérieure à un seuil $\tau>50~\%$. Toutefois, cette valeur n'est pas toujours la meilleure à utiliser.

Prenons l'exemple d'une girafe qui veut déterminer si l'animal qu'elle voit au loin est bien un lion. Si $P(\text{lion})\le 10~\%$, la girafe pense qu'elle voit probablement un autre type d'animal et qu'elle n'est pas en danger. Si $P(\text{lion})\gt 10~\%$, elle pense qu'elle est en danger et doit s'enfuir. Ainsi, la girafe n'a pas besoin d'attendre que le lion soit à 20 mètres d'elle
pour avoir $P(\text{lion})\gt 50~\%$ pour s'enfuir. À ce moment-là, il serait déjà trop tard!
<!-- #endregion -->

<!-- #region id="c25c3bea" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/giraffes.jpeg" width="400" />
    <div>
    <font size="0.5">Image Source: https://cdn.pixabay.com/photo/2017/10/30/21/53/giraffes-2903626_960_720.jpg</font>
    </div>
</div>
<!-- #endregion -->

<!-- #region id="4e11380b" -->
Les valeurs des métriques précédentes sont fortement dépendantes de la valeur de seuil $\tau$ utilisée dans chaque cas. Il est préférable d'utiliser une approche plus globale où les performances d'un classificateur sont évaluées en tenant compte de toutes les valeurs possibles de seuil $\tau$. Nous allons discuter de deux nouvelles métriques *globales*
couramment utilisées pour comparer entre elles les performances de multiples classificateurs. Chacune consiste à mesurer l'aire sous la courbe (*area under curve* ou *AUC*) combinant deux des métriques précédentes entre lesquelles il est souvent difficile de choisir.
<!-- #endregion -->

<!-- #region id="85daeead" -->
## <a id=la-courbe-roc-ireceiver-operator-characteristici>La courbe ROC (*Receiver Operator Characteristic*)</a>
<!-- #endregion -->

<!-- #region id="58ddec82" -->
La fonction d’efficacité du récepteur montre comment varie le taux de vrais positifs
(TVP) en fonction du taux de faux positifs (TFP). Ces éléments correspondent aux équations suivantes:

$$
\begin{align}
\text{TVP} &= \dfrac {\text{VP}}{\text{VP} + \text{FN}}\\
\text{TFP} &= \dfrac {\text{FP}}{\text{FP} + \text{VN}}.
\end{align}
$$
<!-- #endregion -->

<!-- #region id="728ed0b9" -->
La figure ci-dessous montre les courbes ROC pour divers classificateurs. Le classificateur idéal est celui qui ne fait aucune erreur, c'est-à-dire, $\text{FN}=\text{FP}=0$, et donc $\text{TFP}=\text{FPR}=0$ et $\text{TVP}=\text{TPR}=1$. Cela correspond à la position du point bleu, en haut, à gauche. Plus la courbe ROC d'un classificateur se rapproche de ce point, meilleures sont ses performances. Cela se traduit également par une plus grande aire sous la courbe. Lorsque l'on doit choisir entre plusieurs classificateurs, celui avec la plus grande aire sous la courbe doit être sélectionné.

> À noter que la ligne en pointillés rouge correspond à un classificateur générant un résultat parfaitement aléatoire, soit $50~\%$ de classe 0 et $50~\%$ de classe 1.

<!-- #endregion -->

<!-- #region id="86bf2ea3" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/roc-curve.png" width="400" />
    <div>
    <font size="0.5">Image Source: https://en.wikipedia.org/wiki/Receiver_operating_characteristic</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="be0463b8" -->
## <a id=la-courbe-précision-rappel>La courbe Précision-Rappel</a>
<!-- #endregion -->

<!-- #region id="cfcd8e69" -->
Cette courbe est similaire à la précédente, sauf qu'elle utilise la précision et le rappel plutôt
que les taux de vrais et de faux positifs.

La figure suivante montre comment varient conjointement le rappel et la précision. Le code
de couleur indique la valeur de seuil de probabilité $\tau \in [0,1]$ correspondante à
chaque couple de valeurs (rappel, précision).
<!-- #endregion -->

<!-- #region id="6dcc5f1c" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/pr-curve.png" width="500" />
    <div>
    <font size="0.5">Image Source: https://stackoverflow.com/questions/25020788/in-r-calculate-area-under-precision-recall-curve-aupr</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="0f853f08" -->
Le classificateur idéal est celui qui ne fait aucune erreur, c'est-à-dire, $\text{FN}=\text{FP}=0$ et donc $\text{Rappel} = \text{Précision} = 1$.
Cela correspond au coin supérieur droit. Plus la courbe Précision-Rappel d'un classificateur se rapproche de ce point,
meilleures sont ses performances. Cela se traduit, à nouveau, par une plus grande aire sous la courbe. Comme pour
la courbe ROC, si l'on doit choisir entre plusieurs classificateurs, on doit sélectionner celui avec la plus grande
aire sous la courbe Précision-Rappel.

<!-- #endregion -->

<!-- #region id="4b640325" -->
Maintenant, affichons l'aire sous la courbe ROC pour chaque espèce d'iris. On utilise à nouveau une approche un contre tous. Par exemple dans le cas de l'espèce Setosa, il y a deux classes: Setosa et non Setosa.
<!-- #endregion -->



In [ ]:
print("Classe       Aire sous la courbe ROC")
for i in range(n_classes):
    auc = roc_auc_score(y_test == i, y_pred == i)
    print("%s   %0.2f" % (noms[i].ljust(max_length), auc))



<!-- #region id="aeb7b7c9" -->
On observe une valeur de $100~\%$ pour l'espèce setosa! Le classificateur est particulièrement
performant avec cette espèce de fleur. Ce n'est pas le cas avec les deux autres espèces;
néanmoins $95~\%$ et $97~\%$ ne sont pas si mal!
<!-- #endregion -->

<!-- #region id="d661a586" -->
## <a id=quelle-métrique-globale-choisir-pour-sélectionner-un-classificateur-parmi-plusieurs>Quelle métrique globale choisir pour sélectionner un classificateur parmi plusieurs?</a>
<!-- #endregion -->

<!-- #region id="35b531e1" -->
Voilà une question souvent posée. La réponse tient au problème du débalancement des classes dans
les jeux de données. Ce problème est discuté plus en détail dans un module sur la méthodologie.

Soit un jeu de données où la classe 1 a la plus faible probabilité (p. ex. présence de cancer).
Il est recommandé de choisir le classificateur ayant la valeur la plus grande de la métrique suivante:

- jeux de données balancés: aire sous la courbe ROC,
- jeux de données déséquilibrés: aire sous la courbe Précision-Rappel.
<!-- #endregion -->
